<a href="https://colab.research.google.com/github/Hongjieun123/deeplearning-project/blob/master/vgg16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
auth.authenticate_user()


# 구글 드라이브 mount 
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
# 1. 홈 디렉토리에 datasets 폴더 생성 후, 그 안에 cats_and_dogs_small 폴더를 생성 합니다.
# 2. cats_and_dogs_small 안에 train, test, validation 폴더가 위치 하도록 압축을 풉니다.
%cd gdrive/My Drive/datasets/cats_and_dogs_small

/content/gdrive/My Drive/datasets/cats_and_dogs_small


In [0]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator    #ImageDataGenerator  구글드라이브 사진 가져와 쓰겠다. 

# 모든 이미지를 1/255로 스케일을 조정합니다
#0~255/255 -> 0~1사이로 만듦
#image augmentation -> 같은 사진을 변형해서 여러 샘플을 만들어준다.
# 모든 이미지를 1/255로 스케일을 조정합니다
#내가 정해놓은 속성에 맞게 불러오겠다.
# (batch, height, width, channel)
# Numpy N-dims Array (60000, 28, 28, 1) batch_size = 4 --> (4, 28, 28, 1)
train_datagen = ImageDataGenerator(rescale = 1./255, 
                                   shear_range = 0.2, 
                                   zoom_range = 0.2, 
                                   horizontal_flip = True)
# Batch_size = 4 -> (4, 28, 28, 1)  
test_datagen = ImageDataGenerator(rescale=1./255)

base_dir = '/content/gdrive/My Drive/datasets/cats_and_dogs_small'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')


train_generator = train_datagen.flow_from_directory(
        # 타깃 디렉터리
        train_dir,
        # 모든 이미지를 224 × 224 크기로 바꿉니다(왜?? 입력데이터가 224x224로 고정되어 있기 때문)
        target_size=(224, 224),  #이미지 사이즈
        batch_size=20,   #몇장 가져올건지 (random하게 4장)
        # binary_crossentropy 손실을 사용하기 때문에 이진 레이블이 필요합니다
        class_mode='categorical') #가지고 온 데이터의 클래스

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(224, 224),
        batch_size=4,
        class_mode='categorical')



In [4]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

# 모든 이미지를 1/255로 스케일을 조정합니다
# (batch, height, width, channel)
# Numpy N-dims Array (60000, 28, 28, 1) batch_size = 4 --> (4, 28, 28, 1)
train_datagen = ImageDataGenerator(rescale = 1./255, 
                                   shear_range = 0.2, 
                                   zoom_range = 0.2, 
                                   horizontal_flip = True)
# Batch_size = 4 -> (4, 28, 28, 1)
# Augmentation: Shear, Zoom, Flip

test_datagen = ImageDataGenerator(rescale=1./255)
# Augmentation X , Scale [0~1]

base_dir = '/content/gdrive/My Drive/datasets/cats_and_dogs_small'

train_dir = os.path.join(base_dir, 'train')
test_dir  = os.path.join(base_dir, 'test')
validation_dir = os.path.join(base_dir, 'validation')

train_generator = train_datagen.flow_from_directory(
        # 타깃 디렉터리
        train_dir,
        # 모든 이미지를 224 × 224 크기로 바꿉니다
        target_size=(224, 224),
        batch_size=4,
        # binary_crossentropy 손실을 사용하기 때문에 이진 레이블이 필요합니다
        class_mode='categorical')
# Binary 0 ~ 1
# Categorical [1, 0] == 0
#             [0, 1] == 1


validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(224, 224),
        batch_size=4,
        class_mode='categorical')

Using TensorFlow backend.


Found 121 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


In [5]:
for data_batch, labels_batch in train_generator:
    print ('Batch size:', data_batch.shape)
    print ('Label size:', labels_batch.shape)
    break

# One-Hot encoding Example
# 2개 0, 1
# 0: [1, 0]
# 1: [0, 1]

Batch size: (4, 224, 224, 3)
Label size: (4, 2)


In [6]:
print (train_generator.class_indices)

{'cats': 0, 'dogs': 1}


In [7]:
#  This code cell shows how to utilize VGG16 model by combining Dense layer at the end of the network 
from keras.layers import Input, Dense, GlobalAveragePooling2D, Flatten
from keras.models import Model

from keras.applications.vgg16 import VGG16
from keras import layers
from keras import models

VGGNet = VGG16()
VGGNet.summary()

553467904/553467096 [==============================] - 34s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
___________________________________________________________

In [8]:
# We will not update VGG pre-trained model, only added Dense layers will be trained from the scratch
for layer in VGGNet.layers:
  layer.trainable = False  #학습 가능 여부 , 업데이트가 안되도록 막음. (frozen)
  
Feature_Flatten = VGGNet.get_layer('flatten').output

dense = Dense(256, name='encoding', activation='relu')(Feature_Flatten)
predictions = Dense(2, activation='softmax', name="prediction")(dense)

New_VGGmodel = Model(inputs=VGGNet.input, outputs=predictions)
New_VGGmodel.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

New_VGGmodel.summary()


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [9]:
history = New_VGGmodel.fit_generator(
    train_generator,
    # validation_data=validation_generator, 
    # validation_steps=5,
    # 20장 / batch_size 4 = 5
    steps_per_epoch=50,
    # 60장 / batch_size 4 = 15
    epochs=5
    # 전체 반복 횟수 입니다.
)

Epoch 1/5
15/15 [==============================] - 34s 2s/step - loss: 8.2538 - accuracy: 0.4912
Epoch 2/5
15/15 [==============================] - 31s 2s/step - loss: 2.3066 - accuracy: 0.6000
Epoch 3/5
15/15 [==============================] - 32s 2s/step - loss: 1.0123 - accuracy: 0.7500
Epoch 4/5
15/15 [==============================] - 29s 2s/step - loss: 1.1738 - accuracy: 0.7018
Epoch 5/5
15/15 [==============================] - 32s 2s/step - loss: 1.0319 - accuracy: 0.8167


In [10]:
pip install flickrapi

     |████████████████████████████████| 61kB 2.8MB/s 
